In [18]:
from urllib.request import urlopen
from zipfile import ZipFile
from io import BytesIO
import pandas as pd

headers = {'User-Agent': 'Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'}

url = 'https://files.digital.nhs.uk/assets/ods/current/egpcur.zip'
response = urlopen(url=url,  timeout=120)

myzip = ZipFile(BytesIO(response.read()))
myzip.namelist()
myzip.extractall('ZIP')

df = pd.read_csv('ZIP/egpcur.csv', header=None)


df.rename(columns={0: "Code", 1: "Name", 2: "z"})

df



C:\Users\lilyh\AppData\Local\Temp\ipykernel_8828\2564219521.py:15: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ZIP/egpcur.csv', header=None)


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,G0102005,ALLEN EB,Y11,QAL,"FIRCROFT, LONDON ROAD",ENGLEFIELD GREEN,EGHAM,SURREY,NaN,TW20 0BS,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
1,G0102926,ANDERSON MG,Y61,QUE,LENSFIELD MEDICAL PRAC.,48 LENSFIELD ROAD,CAMBRIDGE,CAMBRIDGESHIRE,NaN,CB2 1EH,...,01223 651020,NaN,NaN,NaN,1,NaN,06H,NaN,NaN,NaN
2,G0105912,ADLER S,Y56,QMJ,682 FINCHLEY ROAD,GOLDERS GREEN,LONDON,NaN,NaN,NW11 7NP,...,020 84559994,NaN,NaN,NaN,1,NaN,93C,NaN,NaN,NaN
3,G0107031,ATTWOOD DC,Y62,QOP,GREAT LEVER HEALTH CENTRE,"RUPERT STREET,GREAT LEVER",BOLTON,LANCASHIRE,NaN,BL3 6RN,...,01204 462141,NaN,NaN,NaN,1,NaN,00T,NaN,NaN,NaN
4,G0107725,ALEXANDER PJ,Y01,QDF,10 WEST END,SWANLAND,HUMBERSIDE,NaN,NaN,HU14 3PE,...,0482 633570,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123768,G9996043,UNIDENTIFIED GPS,W00,Q99,NORTH WALES HA,PRESWYLFA,HENDY ROAD,MOLD FLINTSHIRE,NaN,CH7 1PZ,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
123769,G9996050,UNIDENTIFIED GPS,W00,Q99,MORGANNWG HA,41 HIGH STREET,SWANSEA,WEST GLAMORGAN,NaN,SA1 1LT,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
123770,G9996067,COMMITTEES LOCUM,W00,QW3,DEPUTISING SERVICES,POWYS,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
123771,G9996074,COMMITTEES LOCUM,W00,QW2,DEPUTISING SERVICES,SOUTH-GLAMORGAN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN


In [12]:
df